In [1]:
import numpy as np
import pandas as pd
import nltk.corpus
from nltk.corpus import stopwords
from cleantext import clean
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import nltk
import re
import math
from collections import Counter
import json


# 2. Search Engine

### Pre-processing

In [2]:
stop = stopwords.words('english') #stopwords in english 
ps = PorterStemmer() #takes the stem of words
def clean_text(text): 
    text.lower  #case normalization
    text = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)  #removing unicode characters
    text = " ".join([word for word in text.split() if word not in stop and word.isalpha() == True]) #removing stopwords and 'numerical term
    token = word_tokenize(text) #tokenization
    text = [ps.stem(t) for t in token] #stemming and removing non english words
    return(" ".join(text))

## 2.1. Conjunctive query
### 2.1.1 Create your index!


In [3]:
df_for_last=pd.read_csv(r'C:\Users\PC\Desktop\places.tsv', sep='\t') # reading the places with pandas
df=df_for_last.copy()
#df=df[0:20]#temporary

for x,i in enumerate(df.loc[:,"placeDesc"]): #cleaning all text in  column
    df.loc[:,"placeDesc"][x]=clean_text(i)
    
word_list=[] 
string=""
for i in df.loc[:,"placeDesc"]: #string containing all words in the description descriptions to 
    string=string+i+" "
word_list.append(string) 
word_list=word_list[0].split()# getting wordlist out of the list and splitting all words and putting into a set to remove duplicates
word_list_set=set(word_list)



C:\Users\PC\AppData\Local\Temp\ipykernel_5332\2395749016.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,"placeDesc"][x]=clean_text(i)


### Creating a dictionary 

In [5]:
dictionary={i[1]:f"termId_{i[0]}" for i in enumerate(word_list_set)}#creates a dictionary from the set of words with term ids
dict(list(dictionary.items())[0:10])

{'thecounti': 'termId_0',
 'deadhous': 'termId_1',
 'daub': 'termId_2',
 'hydrant': 'termId_3',
 'poguetook': 'termId_4',
 'oto': 'termId_5',
 'amadeo': 'termId_6',
 'hermosa': 'termId_7',
 'somethingwhich': 'termId_8',
 'olaughlin': 'termId_9'}

### Creating an inverted index

In [6]:
inverted_index={i:[] for i in dictionary.values()}
for x,y in enumerate(df.loc[:,"placeDesc"]):
    for i in y.split():
        if f"document_{x}" not in inverted_index[dictionary[i]]:
            inverted_index[dictionary[i]].append(f"document_{x}")
dict(list(inverted_index.items())[0:5])

{'termId_0': ['document_2123', 'document_2448'],
 'termId_1': ['document_4135'],
 'termId_2': ['document_1852'],
 'termId_3': ['document_980',
  'document_2907',
  'document_4203',
  'document_4739',
  'document_4926',
  'document_5407',
  'document_5885'],
 'termId_4': ['document_3876']}

#### Saving inverted index into a json file

In [7]:
with open(r"C:\Users\PC\Desktop\data.json", 'w') as fp: 
    json.dump(dictionary, fp)

### 2.1.2 Execute the query

In [8]:
query_input='new york'
#query_input=input(print("query"))

documents which include all words in the query input

In [9]:
return_list=[]
for i in query_input.split():
    term=dictionary[i]
    return_list.append(set(inverted_index[term]))
inter=[i for i in set.intersection(*return_list)]
inter

['document_2265',
 'document_3645',
 'document_1837',
 'document_2644',
 'document_1180',
 'document_5141',
 'document_6013',
 'document_5829',
 'document_1581',
 'document_5012',
 'document_1766',
 'document_2506',
 'document_61',
 'document_502',
 'document_5500',
 'document_2056',
 'document_2813',
 'document_4417',
 'document_4537',
 'document_4958',
 'document_1078',
 'document_2829',
 'document_4941',
 'document_214',
 'document_470',
 'document_3262',
 'document_251',
 'document_701',
 'document_1781',
 'document_4441',
 'document_5095',
 'document_2264',
 'document_1173',
 'document_149',
 'document_4762',
 'document_1803',
 'document_4682',
 'document_5952',
 'document_4967',
 'document_6010',
 'document_5321',
 'document_1132',
 'document_4101',
 'document_2744',
 'document_2071',
 'document_5204',
 'document_5286',
 'document_1105',
 'document_813',
 'document_843',
 'document_576',
 'document_5028',
 'document_4011',
 'document_1214',
 'document_5567',
 'document_5448',
 'd

### results of documents with query 

In [10]:
df_for_last.loc[[int(i.split("_")[1]) for i in inter],["placeName","placeDesc","placeURL"]]

,placeName,placeDesc,placeURL
2265,Calistoga Petrified Forest,The same turbulent forces that heat the waters...,https://www.atlasobscura.com/places/calistoga-...
3645,Haymarket Square,It’s sometimes said that Chicago invented the ...,https://www.atlasobscura.com/places/haymarket-...
1837,The Smile Face Museum,A shrine to the iconic yellow grin that has ac...,https://www.atlasobscura.com/places/the-smile-...
2644,Toynbee Tiles,There are a few simple rules for walking in Ne...,https://www.atlasobscura.com/places/toynbee-tiles
1180,Central Railroad of New Jersey Terminal,The Romanesque red brick structure of the Cent...,https://www.atlasobscura.com/places/central-ra...
...,...,...,...
2141,The Carousel in Prospect Park,It’s rare that one can actually climb on a 100...,https://www.atlasobscura.com/places/the-carous...
1646,New York Society Library,The New York Society Library occupies a 5-stor...,https://www.atlasobscura.com/places/new-york-s...
3036,"Paul Bunyan Statue, Bangor","Towering over Main Street since 1959, a 31-foo...",https://www.atlasobscura.com/places/paul-bunya...
5539,Bensonhurst Statue House,And you thought gnomes were bad. It’s hard to ...,https://www.atlasobscura.com/places/bensonhurs...


# 2.2) Conjunctive query & Ranking score


## 2.2.1) Inverted index


In [11]:
word_list[:10]

['the',
 'first',
 'new',
 'york',
 'citi',
 'subway',
 'built',
 'oper',
 'interborough',
 'rapid']

#### TF's of documents

In [12]:
TFS=[]
for i in df.loc[:,"placeDesc"]:
    tfs={}
    dene2=i.split()
    for x in dene2:
        tfs[x]=dene2.count(x)/len(dene2)
    TFS.append(tfs)
TFS
document_TFs=pd.DataFrame(TFS)
document_TFs = document_TFs.fillna(0)

document_TFs[:10]


,the,first,new,york,citi,subway,built,oper,interborough,rapid,...,matryoshka,anotherwal,cityanoth,inconceiv,lappool,getawaywhil,overh,steamdrunk,harcho,tabaka
0,0.010204,0.005102,0.015306,0.015306,0.030612,0.005102,0.005102,0.005102,0.005102,0.005102,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.003390,0.003390,0.000000,0.000000,0.006780,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.017544,0.000000,0.000000,0.000000,0.005848,0.000000,0.011696,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.010526,0.002632,0.000000,0.000000,0.002632,0.000000,0.002632,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.011236,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.000000,0.000000,0.000000,0.000000,0.017751,0.005917,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.020833,0.000000,0.000000,0.000000,0.010417,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.016949,0.000000,0.033898,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
new_inverted_index={i:[] for i in dictionary.values()}#empty inverted indexes
for x,y in enumerate(df.loc[:,"placeDesc"]):
    splitted=y.split()
    splitted_tekrarlamaz=set(splitted)
    for i in splitted_tekrarlamaz:         
        new_inverted_index[dictionary[i]].append([x,f"tfidf={document_TFs.loc[x][i]}"])
new_inverted_index
dict(list(new_inverted_index.items())[0:5])

{'termId_0': [[2123, 'tfidf=0.005988023952095809'],
  [2448, 'tfidf=0.008547008547008548']],
 'termId_1': [[4135, 'tfidf=0.011428571428571429']],
 'termId_2': [[1852, 'tfidf=0.007633587786259542']],
 'termId_3': [[980, 'tfidf=0.004132231404958678'],
  [2907, 'tfidf=0.0375'],
  [4203, 'tfidf=0.005405405405405406'],
  [4739, 'tfidf=0.03496503496503497'],
  [4926, 'tfidf=0.006329113924050633'],
  [5407, 'tfidf=0.0024752475247524753'],
  [5885, 'tfidf=0.04504504504504504']],
 'termId_4': [[3876, 'tfidf=0.0022222222222222222']]}

## 2.2.2) Execute the query


#### Creating relevant dataframe for calculating similarity of query to the document

In [14]:
# for every word in query its term_ID is learned from the dictionary and corresponding document numbers in inverted index are
#added to the "rdocs".These are the documents which has the query words in them.

query_input="city subway"
query_input=clean_text(query_input)
return_list=[]
for i in query_input.split():
    term=dictionary[i]
    return_list.append(set(inverted_index[term]))
return_list=[i for i in set.intersection(*return_list)]

rdocs=return_list
rdocs_numbers=[int(i.split("_")[1]) for i in rdocs]


#### Number of documents that has the query words

In [15]:
dictionary_of_words_query={}
for word in query_input.split():
    count=0
    for document in df.loc[:,"placeDesc"]:
        if word in document.split():
            count+=1
    dictionary_of_words_query[word]=count
dictionary_of_words_query

    
    

{'citi': 1579, 'subway': 51}

#### TF-IDF's of relevant words in documents

In [16]:
rTFIDFS={}
for i in query_input.split():
    rTFIDFS[i]=[]
    tfidfs=[]
    dene2=i.split()
    for x in rdocs_numbers:
        rTFIDFS[i].append(df.loc[:,"placeDesc"][x].count(i)/len(df.loc[:,"placeDesc"][x].split())*(1+math.log(len(df.loc[:,"placeDesc"])/dictionary_of_words_query[i])))
document_TFIDFs=pd.DataFrame(rTFIDFS, index=rdocs_numbers)
document_TFIDFs[:5]


,citi,subway
473,0.010060,0.123966
591,0.034727,0.042791
1017,0.014651,0.072210
253,0.021907,0.107978
3465,0.043952,0.090263


In [17]:
#new dataframe for documents including query

relevant_df=document_TFIDFs.loc[rdocs_numbers][query_input.split()]


In [18]:
relevant_df[:10]

,citi,subway
473,0.010060,0.123966
591,0.034727,0.042791
1017,0.014651,0.072210
253,0.021907,0.107978
3465,0.043952,0.090263
5091,0.023324,0.057481
72,0.009568,0.070736
701,0.012671,0.031226
5559,0.046882,0.202188
4095,0.012146,0.029932


#### Calculating the similarity between query and documents

In [19]:
dictionary_of_words_query={}
for word in relevant_df:
    count=0
    for document in df.loc[:,"placeDesc"]:
        if word in document.split():
            count+=1
    dictionary_of_words_query[word]=count
dictionary_of_words_query

    
    

{'citi': 1579, 'subway': 51}

In [20]:
#TFIDF of query
query_tfIDFs=[]
for x in query_input.split():
    query_tfIDFs.append(query_input.split().count(x)/len(query_input.split())*  (1+math.log(len(query_input.split())/dictionary_of_words_query[x])))
query_tfIDFs


[-2.8356999168478483, -1.1193392260821902]

In [21]:
#||Query||
add=0
for i in query_tfIDFs:
    add+=i*i    
Query_L=math.sqrt(add)   

#Similarities of all documents
similarity_list=[]
for i in relevant_df.index:
    add1=0
    for x in relevant_df.loc[i]:
        add1+=x*x    
    Document_L=math.sqrt(add1)   
    similarity_list.append((np.dot(relevant_df.loc[i],query_tfIDFs))/Query_L*Document_L)
    #similarity_list.append([i,(np.dot(relevant_df.loc[i],query_tfIDFs))/Query_L*Document_L])

sorted(similarity_list)

[-0.02445871031268192,
 -0.015811693165782622,
 -0.011049632472868188,
 -0.010911067776051414,
 -0.010515115156533101,
 -0.007719684344047242,
 -0.007431529116321192,
 -0.007128590260265148,
 -0.006824800807895051,
 -0.006613167168863311,
 -0.006017320694751093,
 -0.005858536317031171,
 -0.005626794806503869,
 -0.004817277206611101,
 -0.0029953199240436105,
 -0.002957584020168596,
 -0.0027601437163254604,
 -0.0026550008591815586,
 -0.0026459876968171585,
 -0.0024891309036560124,
 -0.0024167433022540155,
 -0.0017938036257117967,
 -0.0017213831924616009,
 -0.001716235035388707,
 -0.0016891013119141177,
 -0.0013917178285779864,
 -0.0009615322323478267,
 -0.0007835258561854943,
 -0.0007593714691120671,
 -0.0007199165730072901,
 -0.00037335949582240665,
 -0.0003255614664248508]

#### results

In [22]:
sonuc_new=df_for_last.loc[rdocs_numbers,["placeName","placeDesc","placeURL"]]
sonuc_new.insert(3,"similarity",sorted(similarity_list))

#add similarity list at the end of the dataframe

In [23]:
sonuc_new=sonuc_new.sort_values("similarity",ascending=False)
sonuc_new

,placeName,placeDesc,placeURL,similarity
2779,London Underground Labyrinths,"In the 2000s, London’s government asked artist...",https://www.atlasobscura.com/places/london-und...,-0.000326
1234,Foundry Branch Trolley Trestle Ruins,"Go for a hike through Glover-Archbold park, ju...",https://www.atlasobscura.com/places/foundry-br...,-0.000373
5943,Site of the Beach Pneumatic Subway,Alfred Beach built this pneumatic tunnel to sh...,https://www.atlasobscura.com/places/beach-pneu...,-0.000720
4568,Moscow Metro Stations,"Subway stations are often sordid spots, the da...",https://www.atlasobscura.com/places/moscow-met...,-0.000759
5446,The Marriage of Money and Real Estate,Little green men stand ready to invade the wes...,https://www.atlasobscura.com/places/the-marria...,-0.000784
2168,Life Underground Sculptures,"Each day, thousands of subway riders at the 14...",https://www.atlasobscura.com/places/life-under...,-0.000962
0,City Hall Station,The first New York City subway was built and o...,https://www.atlasobscura.com/places/city-hall-...,-0.001392
2128,The Daily News Building Globe,The Daily News Building on Manhattan’s East 42...,https://www.atlasobscura.com/places/the-daily-...,-0.001689
5144,Streetcar Deck of the Detroit-Superior Bridge,The Detroit-Superior Bridge in downtown Clevel...,https://www.atlasobscura.com/places/streetcar-...,-0.001716
7,Underground Tunnels of Los Angeles,While the rest of the country was forced to go...,https://www.atlasobscura.com/places/undergroun...,-0.001721
